<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The Cartesian Schwarzschild Hamiltonian

## Author: Tyler Knowles, Siddharth Mahesh
### Formatting improvements courtesy Brandon Clark

## This module documents the Schwarzschild Hamiltonian in Cartesian Coordinates for Testing the Derivative Routines.


**Notebook Status:** <font color='red'><b> In Progress </b></font>

~~**Validation Notes:** This module has been validated against the LALSuite [SEOBNRv3/SEOBNRv3_opt code]( https://git.ligo.org/lscsoft/lalsuite.) that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.  That is, the value $H_{\rm real}$ output from this notebook agrees to roundoff error with the value of $H_{\rm real}$ computed by the LALSuite function XLALSimIMRSpinPrecEOBHamiltonian().~~

### NRPy+ Source Code for this module: [SEOBNR_v3_Hamiltonian.py](../edit/SEOBNR/SEOBNR_v3_Hamiltonian.py)

<a id='intro'></a>

## Introduction
$$\label{intro}$$

### The Physical System of Interest

Consider a black hole with mass $M$ and no spin with a non-spinning test particle that evolves under its influence.  The Schwarzschild Hamiltonian $H_{\rm real}$ (defined in [this cell](#hreal)) describes the dynamics of this system.~~; we will define $H_{\rm real}$ as in [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) Section VE.  There, $H_{\rm real}$ is canonically transformed and mapped to an effective Hamiltonian $H_{\rm eff}$ (defined in [this cell](#heff)) describing the motion of a test particle of mass $\mu$ (defined in [this cell](#mu)) and spin ${\bf S}^{*}$ (defined in [this cell](#sstar)) moving in a defomred Kerr background.~~  Here we seek to break up $H_{\rm real}$ and document the terms in such a way that the resulting Python code can be used to numerically evaluate $H_{\rm real}$.

We write $H_{\rm real}$ in terms of Cartesian quasi-isotropic coordinates $x$, $y$, and $z$ ~~(see [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) Section III).  The spatial coordinates $r$, $\theta$, and $\phi$ referenced throughout are [Boyer-Lindquist coordinates](https://en.wikipedia.org/wiki/Boyer%E2%80%93Lindquist_coordinates) (see [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) Section IV).~~

Please note that throughout this notebook we adpot the following conventions:

1. $c = 1$ where $c$ is the speed of light in a vacuum,
1. spacial tensor indicies are denoted by lowercase Latin letters,
1. repeated indices indicate Einstein summation notation, and
~~1. we normalize $M=1$ in all terms except for $\eta$ and $\mu$ for agreement with LALSuite.  Nonetheless, $M$ appears in other text cells for comparison with the cited literature.~~

Running this notebook to completion will generate a file called Hreal_on_bottom.py.  This file contains the Python function compute_Hreal(), which takes as input $M$ (in solar masses), ~~the value of the [Euler-Mascheroni constant](https://en.wikipedia.org/wiki/Euler%E2%80%93Mascheroni_constant), a tortoise coordinate,~~ and values for all six dynamic variables (3 components of the separation vector, and three components of the momentum vector)~~, and three spin components of each compact object).  Note that the spin components should be dimensionless.~~

### Citations
Throughout this module, we will refer to
* [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010,
* [Barausse and Buonanno (2011)](https://arxiv.org/abs/1107.2904) as BB2011,
* [Steinhoff, Hinderer, Buonanno, et al (2016)](https://arxiv.org/abs/1608.01907) as SH2016,
* [Pan, Buonanno, Buchman, et. al. 2010](https://arxiv.org/abs/0912.3466v2) as P2010,
* [Taracchini, Buonanno, Pan, et al (2014)](https://arxiv.org/abs/1311.2544) as T2014,
* [Taracchini, Pan, Buonanno, et al (2012)](https://arxiv.org/abs/1202.0790) as T2012, and
* [Damour, Jaranowski, and Schaefer (2000)](https://arxiv.org/abs/gr-qc/0005034) as D2000.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows:

1. [Step 0](#outputcreation): Creating the output directory for SEOBNR
1. [Step 1](#hreal): The Real Hamiltonian $H_{\rm real}$
1. [Step 2](#heff): The Effective Hamiltonian $H_{\rm eff}$
1. [Step 3](#heff_terms): Terms of $H_{\rm eff}$  
    1. [Step 3.a](#hs): Leading Order Spin Effects $H_{\rm S}$  
    1. [Step 3.b](#hns): The Nonspinning Hamiltonian $H_{\rm NS}$  
    1. [Step 3.c](#hd): The Quadrupole Deformation $H_{\rm D}$
1. [Step 4](#hso): The Spin-Orbit Term $H_{\rm SO}$  
    1. [Step 4.a](#hsoterm1): $H_{\rm SO}$ Term 1  
    1. [Step 4.b](#hsoterm2coeff): $H_{\rm SO}$ Term 2 Coefficient  
    1. [Step 4.c](#hsoterm2): $H_{\rm SO}$ Term 2  
        1. [Step 4.c.i](#hsoterm2a): $H_{\rm SO}$ Term 2a  
        1. [Step 4.c.ii](#hsoterm2b): $H_{\rm SO}$ Term 2b  
        1. [Step 4.c.iii](#hsoterm2c): $H_{\rm SO}$ Term 2c
1. [Step 5](#hss): The Spin-Spin Term $H_{\rm SS}$  
    1. [Step 5.a](#hssterm1): $H_{\rm SS}$ Term 1  
    1. [Step 5.b](#hssterm2coeff): $H_{\rm SS}$ Term 2 coefficient  
    1. [Step 5.c](#hssterm2): $H_{\rm SS}$ Term 2  
    1. [Step 5.d](#hssterm3coeff): $H_{\rm SS}$ Term 3 coefficient  
    1. [Step 5.e](#hssterm3): $H_{\rm SS}$ Term 3
1. [Step 6](#hnsterms): The $H_{\rm NS}$ Terms  
    1. [Step 6.a](#betapsum): $\beta p$ Sum  
    1. [Step 6.b](#alpha): $\alpha$  
    1. [Step 6.c](#hnsradicand): $H_{\rm NS}$ Radicand  
        1. [Step 6.c.i](#gammappsum): $\gamma p$ Sum  
        1. [Step 6.c.ii](#q4): ${\cal Q}_{4}$
1. [Step 7](#hdterms): The $H_{\rm D}$ Terms  
    1. [Step 7.a](#hdcoeff): $H_{\rm D}$ Coefficient  
    1. [Step 7.b](#hdsum): $H_{\rm D}$ Sum  
        1. [Step 7.b.i](#hdsumterm1): $H_{\rm D}$ Sum Term 1  
        1. [Step 7.b.ii](#hdsumterm2): $H_{\rm D}$ Sum Term 2
1. [Step 8](#dotproducts): Common Dot Products
    1. [Step 8.a](#sdotxi): ${\bf S} \cdot \boldsymbol{\xi}$  
    1. [Step 8.b](#sdotv): ${\bf S} \cdot {\bf v}$  
    1. [Step 8.c](#sdotn): ${\bf S} \cdot {\bf n}$  
    1. [Step 8.d](#sdotskerrhat): ${\bf S} \cdot \hat{\bf S}_{\rm Kerr}$  
    1. [Step 8.e](#sstardotn): ${\bf S}^{*} \cdot {\bf n}$
1. [Step 9](#hreal_spin_combos): $H_{\rm real}$ Spin Combination ${\bf S}^{*}$  
    1. [Step 9a](#sstar): ${\bf S}^{*}$  
    1. [Step 9b](#deltasigmastar): $\Delta_{\sigma^{*}}$  
    1. [Step 9c](#sigmastarcoeff): $\sigma^{*}$ Coefficient  
        1. [Step 9c i](#sigmastarcoeffterm1): $\sigma^{*}$ Coefficient Term 1  
        1. [Step 9c ii](#sigmastarcoeffterm2): $\sigma^{*}$ Coefficient Term 2   
    1. [Step 9d](#sigmacoeff): $\sigma$ Coefficient  
        1. [Step 9d i](#sigmacoeffterm1): $\sigma$ Coefficient Term 1  
        1. [Step 9d ii](#sigmacoeffterm2): $\sigma$ Coefficient Term 2  
        1. [Step 9d iii](#sigmacoeffterm3): $\sigma$ Coefficient Term 3
1. [Step 10](#metpotderivs): Derivatives of the Metric Potential  
    1. [Step 10.a](#omegar): $\omega_{r}$  
    1. [Step 10.b](#nur): $\nu_{r}$  
    1. [Step 10.c](#mur): $\mu_{r}$  
    1. [Step 10.d](#omegacostheta): $\omega_{\cos\theta}$  
    1. [Step 10.e](#nucostheta): $\nu_{\cos\theta}$  
    1. [Step 10.f](#mucostheta): $\mu_{\cos\theta}$  
    1. [Step 10.g](#lambdatprm): $\Lambda_{t}^{\prime}$  
    1. [Step 10.h](#omegatildeprm): $\tilde{\omega}_{\rm fd}^{\prime}$
1. [Step 11](#metpots): The Deformed and Rescaled Metric Potentials  
    1. [Step 11.a](#omega): $\omega$  
    1. [Step 11.b](#exp2nu): $e^{2 \nu}$  
    1. [Step 11.c](#btilde): $\tilde{B}$  
    1. [Step 11.d](#brtilde): $\tilde{B}_{r}$  
    1. [Step 11.e](#exp2mu): $e^{2 \tilde{\mu}}$  
    1. [Step 11.f](#jtilde): $\tilde{J}$  
    1. [Step 11.g](#q): $Q$  
        1. [Step 11.g.i](#drsipn2): $\frac{ \Delta_{r} }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2}$  
        1. [Step 11.g.ii](#qcoeff1): Q Coefficient 1  
        1. [Step 11.g.iii](#qcoeff2): Q Coefficient 2
1. [Step 12](#tort): Tortoise terms  
    1. [Step 12.a](#pphi): $p_{\phi}$  
    1. [Step 12.b](#pdotvr): $\hat{\bf p} \cdot {\bf v} r$  
    1. [Step 12.c](#pdotn): $\hat{\bf p} \cdot {\bf n}$  
    1. [Step 12.d](#pdotxir): $\hat{\bf p} \cdot \boldsymbol{\xi} r$  
    1. [Step 12.e](#hatp): $\hat{\bf p}$  
    1. [Step 12.f](#prt): prT  
    1. [Step 12.g](#csi2): csi2  
    1. [Step 12.h](#csi1): csi1  
    1. [Step 12.i](#csi): csi
1. [Step 13](#metric): Metric Terms  
    1. [Step 13.a](#lambdat): $\Lambda_{t}$  
    1. [Step 13.b](#deltar): $\Delta_{r}$  
    1. [Step 13.c](#deltat): $\Delta_{t}$  
    1. [Step 13.d](#deltatprm): $\Delta_{t}^{\prime}$  
    1. [Step 13.e](#deltau): $\Delta_{u}$  
        1. [Step 13.e.i](#deltaubar): $\bar{\Delta}_{u}$  
        1. [Step 13.e.ii](#deltaucalib): $\Delta_{u}$ Calibration Term  
        1. [Step 13.e.iii](#calib_coeffs): Calibration Coefficients  
        1. [Step 13.e.iv](#k): $K$  
    1. [Step 13.f](#omegatilde): $\tilde{\omega}_{\rm fd}$  
    1. [Step 13.g](#dinv): $D^{-1}$
1. [Step 14](#coord): Terms Dependent on Coordinates  
    1. [Step 14.a](#usigma): $\Sigma$  
    1. [Step 14.b](#w2): $\varpi^{2}$   
    1. [Step 14.d](#sin2theta): $\sin^{2}\theta$  
    1. [Step 14.e](#costheta): $\cos\theta$
1. [Step 15](#vectors): Important Vectors  
    1. [Step 15.a](#v): ${\bf v}$  
    1. [Step 15.b](#xi): $\boldsymbol{\xi}$  
    1. [Step 15.c](#e3): ${\bf e}_{3}$  
    1. [Step 15.d](#n): ${\bf n}$
1. [Step 16](#spin_combos): Spin Combinations $\boldsymbol{\sigma}$, $\boldsymbol{\sigma}^{*}$, and ${\bf S}_{\rm Kerr}$   
    1. [Step 16.a](#a): $a$  
    1. [Step 16.b](#skerrhat): $\hat{\bf S}_{\rm Kerr}$  
    1. [Step 16.c](#skerrmag): $\left\lvert {\bf S}_{\rm Kerr} \right\rvert$  
    1. [Step 16.d](#skerr): ${\bf S}_{\rm Kerr}$  
    1. [Step 16.e](#sigma): $\boldsymbol{\sigma}$  
    1. [Step 16.f](#sigmastar): $\boldsymbol{\sigma}^{*}$
1. [Step 17](#fundquant): Fundamental Quantities  
    1. [Step 17.a](#u): $u$  
    1. [Step 17.b](#r): $r$  
    1. [Step 17.c](#eta): $\eta$  
    1. [Step 17.d](#mu): $\mu$  
    1. [Step 17.e](#m): $M$
1. [Step 18](#validation): Validation
1. [Step 19](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='outputcreation'></a>

# Step 0: Creating the output directory for Schwarzschild \[Back to [top](#toc)\]
$$\label{outputcreation}$$

First we create the output directory for Schwarzschild (if it does not already exist):

In [1]:
import numpy as np
import difflib, sys, os

nrpy_dir_path = os.path.join("..") # TYLERK: Remove these three lines when moving to main NRPy+ directory
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)


import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Schwarzschild"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

<a id='heff'></a>

# Step 1: The Hamiltonian $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{heff}$$

The effective Hamiltonian $H_{\rm real}$ ~~is given by [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.70)~~:

\begin{equation*}
    H_{\rm eff} = \underbrace{ \alpha \sqrt{ \mu^{2} + \gamma^{ij} p_{i} p_{j} } }_{ H_{\rm NS} } 
\end{equation*}

Here, ~~,$H_{\rm S}$ (considered further in [this cell](#hs)) denotes leading order effects of spin-spin and spin-orbit coupling,~~ $H_{\rm NS}$ (considered further in [this cell](#hns)) is the Hamiltonian for a nonspinning test particle~~, and $H_{\rm D}$ (considered further in [this cell](#hd)) describes quadrupole deformation of the coupling of the particle's spin with itself to leading order.  [T2014](https://arxiv.org/abs/1311.2544) adds to $H_{\rm eff}$ a 3PN spin-spin term given by~~

\begin{equation*}
    \frac{d_{\rm SS} \eta }{ r^{4} } \left( {\bf S}_{1}^{2} + {\bf S}_{2}^{2} \right)
\end{equation*}

~~where $d_{\rm SS}$ is an adjustable parameter determined by fitting to numerical relativity results.  We take $u \equiv \frac{1}{r}$ (as described in [this cell](#u)) and define $\eta$ in [this cell](#eta).~~

In [2]:
%%writefile  $Ccodesdir/Hamiltonian-Hreal_on_top.txt
Heff = Hns

Overwriting Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='heff_terms'></a>

# Step 2: Terms of $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{heff_terms}$$

In this step, we break down ~~each of the terms $H_{\rm S}$ (defined in [this cell](#hs)),~~$H_{\rm NS}$ (defined in [this cell](#hns))~~, and $H_{\rm D}$ (defined in [this cell](#hd)) in $H_{\rm eff}$ (defined in [this cell](#heff)).

<a id='hns'></a>

## Step 2.a: The Nonspinning Hamiltonian $H_{\rm NS}$ \[Back to [top](#toc)\]
$$\label{hns}$$

We defined $H_{\rm NS}$ in [this cell](#heff) as

\begin{equation*}
    H_{\rm NS} = \alpha \sqrt{ \smash[b]{ \underbrace{ \mu^{2} + \gamma^{ij} p_{i} p_{j}}_{ H_{\rm NS}\ \rm radicand } } }.
\end{equation*}

We compute ~~,$\beta\ p$ sum in [this cell](#betapsum),~~ $\alpha$ in [this cell](#alpha), and $H_{\rm NS}$ radicand in [this cell](#hnsradicand).

In [3]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
Hns = alpha*sp.sqrt(Hnsradicand)

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='hnsterms'></a>

# Step 3: $H_{\rm NS}$ Terms \[Back to [top](#toc)\]
$$\label{hnsterms}$$

We collect here the terms in $H_{\rm NS}$ (defined in [this cell](#hns)).

<a id='alpha'></a>

## Step 3.a: $\alpha$ \[Back to [top](#toc)\]
$$\label{alpha}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.44), we have
\begin{equation*}
    \alpha = \frac{ 1 }{ \sqrt{ -g^{tt}} },
\end{equation*}

and ~~from [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.36a)~~ we have

\begin{equation*}
    g^{tt} = -\frac{ 1 }{ A }.
\end{equation*}

Therefore

\begin{equation*}
    \alpha = \sqrt{ A }.
\end{equation*}

We define $A$ in [this cell](#deltat)

In [4]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
alpha = sp.sqrt(A)

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='hnsradicand'></a>

## Step 3.b: $H_{\rm NS}$ radicand \[Back to [top](#toc)\]
$$\label{hnsradicand}$$

Recall that we defined $H_{\rm NS}$ radicand in [this cell](#hns) as

\begin{equation*}
    H_{\rm NS}\ {\rm radicand} = \mu^{2} + \underbrace{\gamma^{ij} p_{i} p_{j}}_{\gamma p\ \rm sum}
\end{equation*}

We normalize $\mu = 1$, and define $\gamma p$ sum in [this cell](#gammappsum)

In [5]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
Hnsradicand = 1 + gammappsum

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='gammappsum'></a>

### Step 3.b.i: $\gamma^{ij} p_{i} p_{j}$ \[Back to [top](#toc)\]
$$\label{gammappsum}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.46), we have

\begin{equation*}
    \gamma^{ij} = g^{ij} - \frac{ g^{ti} g^{tj} }{ g^{tt} }.
\end{equation*}

~~Combining this result with [BB2010](https://arxiv.org/abs/0912.3517) Equations 5.36,~~ we have

\begin{equation*}
    \gamma^{r\theta} = \gamma^{r\phi} = \gamma^{\theta r} = \gamma^{\theta\phi} = \gamma^{\phi r} = \gamma^{\phi\theta} = 0
\end{equation*}

and

\begin{align*}
    \gamma^{rr} &= g^{rr} = \frac{ 1 }{ B } \\
    \gamma^{\theta\theta} &= g^{\theta\theta} = \frac{1}{r^2}  \\
    \gamma^{\phi\phi} &= \frac{1}{r^2 \sin^{2} \theta} .
\end{align*}

Therefore,

\begin{align*}
    \gamma^{ij} p_{i} p_{j} &= \gamma^{rr} p_{r} p_{r} + \gamma^{\theta\theta} p_{\theta} p_{\theta} + \gamma^{\phi\phi} p_{\phi} p_{\phi} \\
        &= \frac{ 1 }{ B } p_{r}^{2} + \frac{ 1 }{ r^2 } p_{\theta}^{2} + \frac{ 1 }{ r^2 \sin^{2} \theta } p_{\phi}^{2}.
\end{align*}

We define $B$ in [this cell](#deltar)~~, $\Sigma$ in [this cell](#sigma),~~ $ p_r$ in [this cell](#pdotn), $p_\theta$ in [this cell](#pdotvr), $\sin^{2} \theta$ in [this cell](#sin2theta), $r$ in [this cell](#r), and $p_\phi$ in [this cell](#pdotxir).

In [6]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
gammappsum = pr*pr/B + ptheta*ptheta/r/r + pphi2/r/r/sin2theta

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='tort'></a>

# Step 4: Momentum Terms \[Back to [top](#toc)\]
$$\label{tort}$$

We collect here terms related to the conversion from cartesian coordinates to Schwarzschild coordinates.  ~~Details of the converstion are given in the appendix of [P2010](https://arxiv.org/abs/0912.3466v2).~~

<a id='pdotvr'></a>

## Step 4.a: $p_\theta$ \[Back to [top](#toc)\]
$$\label{pdotvr}$$

We have

\begin{equation*}
    p_\theta = \left( \hat{\theta}_{1} p_{1} + \hat{\theta}_{2} p_{2} + \hat{\theta}_{3} p_{3} \right) r
\end{equation*}

We have $\bf p$ as the canonical momentum vector in cartesian coordinates, and we define $\hat{\boldsymbol{\theta}}$ in [this cell](#xi), and $r$ in [this cell](#r).

In [7]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
ptheta = (thetahat1*p1 + thetahat2*p2 + thetahat3*p3)*r

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='pdotn'></a>

## Step 4.b: $p_r$ \[Back to [top](#toc)\]
$$\label{pdotn}$$

We have

\begin{equation*}
    p_r = \hat{r}_{1} p_{1} + \hat{r}_{2} p_{2} + \hat{r}_{3} p_{3}
\end{equation*}

We define $\hat{\bf r}$ in [this cell](#n).

In [8]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
pr = rhat1*p1 + rhat2*p2 + rhat3*p3

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='pdotxir'></a>

## Step 4.c: $p_\phi$ \[Back to [top](#toc)\]
$$\label{pdotxir}$$

We have

\begin{equation*}
    p_\phi = \left( \hat{\phi}_{1} p_{1} + \hat{\phi}_{2} p_{2} + \hat{\phi}_{3} p_{3} \right) r \sin\theta \\
    p_\phi^2 = \left( \hat{\phi}_{1} p_{1} + \hat{\phi}_{2} p_{2} + \hat{\phi}_{3} p_{3} \right)^2 r^2 \sin^2\theta
\end{equation*}

We define $\hat{\boldsymbol{\phi}}$ in [this cell](#e3), $\sin\theta$ in [this cell](#sin2theta) and $r$ in [this cell](#r).

In [9]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
pphi2 = pphi*pphi*sin2theta
pphi = (phihat1*p1 + phihat2*p2 + phihat3*p3)*r

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='metric'></a>

# Step 5: Metric Terms \[Back to [top](#toc)\]
$$\label{metric}$$

We collect here terms used to define the Schwarzschild metric. ~~See [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.38)--(5.40) and (5.71)--(5.75).~~

<a id='deltar'></a>

## Step 5.a: $B$ \[Back to [top](#toc)\]
$$\label{deltar}$$

We have ~~From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.38),~~

\begin{equation*}
    B = A^{-1}.
\end{equation*}

We define $A$ in [this cell](#deltat)

In [10]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
B = 1/A

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='deltat'></a>

## Step 5.b: $A$ \[Back to [top](#toc)\]
$$\label{deltat}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.71), we have

\begin{equation*}
    A = 1 - 2M/r = 1 - 2u .
\end{equation*}

We define $u$ in [this cell](#u).

In [11]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
A = 1 - 2*u

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='coord'></a>

# Step 6: Terms Dependent on Coordinates \[Back to [top](#toc)\]
$$\label{coord}$$

We collect here terms directly dependend on the coordinates.  See [BB2010](https://arxiv.org/abs/0912.3517) Equations (4.5) and (4.6).

<a id='sin2theta'></a>

## Step 6.a: $\sin^{2} \theta$ \[Back to [top](#toc)\]
$$\label{sin2theta}$$



In [12]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
sin2theta = 1 - costheta*costheta

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='costheta'></a>

## Step 6.b: $\cos \theta$ \[Back to [top](#toc)\]
$$\label{costheta}$$

~~From the discussion in [BB2010](https://arxiv.org/abs/0912.3517) after equation (5.52) (noting that ${\bf e}_{3} = \hat{\bf S}_{\rm Kerr}$),~~

\begin{equation*}
    \cos \theta = \frac{z}{r}
\end{equation*}

We ~~define ${\bf e}_{3}$ in [this cell](#e3)~ and~~ define $r$ in [this cell](#r).

In [13]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
costheta = z*u

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='vectors'></a>

# Step 7: Important Vectors \[Back to [top](#toc)\]
$$\label{vectors}$$

We collect the vectors common for computing $H_{\rm real}$ (defined in [this cell](#hreal)) below.

<a id='xi'></a>

## Step 7.a: $\boldsymbol{\hat{\theta}}$ \[Back to [top](#toc)\]
$$\label{xi}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (3.38), we have

\begin{equation*}
    \boldsymbol{\hat{\theta}} = \boldsymbol{\hat{\phi}} \times \mathbf{\hat{r}}
\end{equation*}

We define ${\boldsymbol{\hat{\phi}}}$ in [this cell](#e3) and ${\mathbf{\hat{r}}}$ in [this cell](#n).

In [14]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
thetahat1 = phihat2*rhat3 - phihat3*rhat2
thetahat2 = -phihat1*rhat3 + phihat3*rhat1
thetahat3 = phihat1*rhat2 - phihat2*rhat1

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='e3'></a>

## Step 7.b: ${\boldsymbol{\hat{\phi}}}$ \[Back to [top](#toc)\]
$$\label{e3}$$

From the discussion in [BB2010](https://arxiv.org/abs/0912.3517) after equation (5.52),

\begin{equation*}
    {\boldsymbol{\hat{\phi}}} = \frac{1}{\sqrt{x^2 + y^2}}(-y,x,0).
\end{equation*}


In [15]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
phihat1 = -y*rhoinv
phihat2 = x*rhoinv
phihat3 = 0
rhoinv = 1/sp.sqrt(x*x + y*y)

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='n'></a>

## Step 7.c: ${\mathbf{\hat{r}}}$ \[Back to [top](#toc)\]
$$\label{n}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (3.37), we have

\begin{equation*}
    {\mathbf{\hat{r}}} = \frac{\bf x }{ r }
\end{equation*}

where ${\bf x} = (x, y, z)$.  We define $r$ in [this cell](#r).

In [16]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
rhat1 = x*u
rhat2 = y*u
rhat3 = z*u

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='fundquant'></a>

# Step 8: Fundamental Quantities \[Back to [top](#toc)\]
$$\label{fundquant}$$

 We collect here fundamental quantities from which we build $H_{\rm real}$ (defined in [this cell](#Hreal)).

<a id='u'></a>

## Step 8.a: $u$ \[Back to [top](#toc)\]
$$\label{u}$$

From the discussion after [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.40),

\begin{equation*}
    u = \frac{ 1 }{ r }.
\end{equation*}

We define $M$ in [this cell](#m) and $r$ in [this cell](#r).

In [17]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
u = 1/r

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='r'></a>

## Step 8.b: $r$ \[Back to [top](#toc)\]
$$\label{r}$$

From the discussion after [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.52),

\begin{equation*}
    r = \sqrt{ x^{2} + y^{2} + z^{2} }.
\end{equation*}

In [18]:
%%writefile -a $Ccodesdir/Hamiltonian-Hreal_on_top.txt
r = sp.sqrt(x*x + y*y + z*z)

Appending to Schwarzschild/Hamiltonian-Hreal_on_top.txt


<a id='validation'></a>

# Step 9: Validation \[Back to [top](#toc)\]
$$\label{validation}$$

The following code cell reverses the order of the expressions output to SEOBNR/Hamiltonian_on_top.txt and creates a Python function to validate the value of $H_{\rm real}$ against the SEOBNRv3 Hamiltonian value computed in LALSuite git commit bba40f21e9 for command-line input parameters

-M 23 -m 10 -f 20 -X 0.01 -Y 0.02 -Z -0.03 -x 0.04 -y -0.05 -z 0.06.

In [19]:
import numpy as np
import difflib, sys, os

# The subterms in the Hamiltonian expression are sometimes written on more than
# one line for readability in this Jupyter notebook.  We first create a file of
# one-line expressions, Hamiltonian-Hreal_one_line_expressions.txt.
with open(os.path.join(Ccodesdir,"Hamiltonian-one_line_expressions.txt"), "w") as output:
    count = 0
    # Read output of this notebook
    for line in list(open("Schwarzschild/Hamiltonian-Hreal_on_top.txt")):
        # Read the first line
        if count == 0:
            prevline=line
        #Check if prevline is a complete expression
        elif "=" in prevline and "=" in line:
            output.write("%s\n" % prevline.strip('\n'))
            prevline=line
        # Check if line needs to be adjoined to prevline
        elif "=" in prevline and not "=" in line:
            prevline = prevline.strip('\n')
            prevline = (prevline+line).replace(" ","")
        # Be sure to print the last line.
        if count == len(list(open("Schwarzschild/Hamiltonian-Hreal_on_top.txt")))-1:
            if not "=" in line:
                print("ERROR. Algorithm not robust if there is no equals sign on the final line. Sorry.")
                sys.exit(1)
            else:
                output.write("%s" % line)
        count = count + 1

# Now reverse the expressions and write them in a function
# This formulation is used to check that we get a reasonable H_real value
with open(os.path.join(Ccodesdir,"Hreal_on_bottom.py"), "w") as output:
    output.write("import numpy as np\ndef compute_Hreal(x=2.129681018601393e+01, y=0.000000000000000e+00, z=0.000000000000000e+00, p1=0.000000000000000e+00, p2=2.335391115580442e-01, p3=-4.235164736271502e-22):\n")
    for line in reversed(list(open("Schwarzschild/Hamiltonian-one_line_expressions.txt"))):
        output.write("    %s\n" % line.rstrip().replace("sp.sqrt", "np.sqrt").replace("sp.Rational",
                                "np.divide").replace("sp.abs", "np.abs").replace("sp.log",
                                "np.log").replace("sp.sign", "np.sign").replace("sp.pi",
                                "np.pi"))
    output.write("    return Heff")


<a id='latex_pdf_output'></a>

# Step 19: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-SEOBNR_Documentation.pdf](Tutorial-SEOBNR_Documentation.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [20]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-SEOBNR_Documentation")

AttributeError: module 'cmdline_helper' has no attribute 'output_Jupyter_notebook_to_LaTeXed_PDF'